In [1]:
import os
import torch

from src.get_datanames import get_filenames

In [2]:
token_dir = "prepared_objects_first_4_levels"
model_ids = get_filenames(root_folder=token_dir)  # Liste aller IDs

In [4]:
model_ids

['1Story__base_000_000_000__checkpoints__final',
 '1Story__compound_090_000_090__checkpoints__final',
 '1Story__x_180_000_000__checkpoints__final',
 '1Story__y_000_180_000__checkpoints__final',
 '1Story__z_000_000_120__checkpoints__final',
 '1Story__z_000_000_240__checkpoints__final',
 'Apple-visemes__base_000_000_000__checkpoints__final',
 'Apple-visemes__compound_090_000_090__checkpoints__final',
 'Apple-visemes__x_180_000_000__checkpoints__final',
 'Apple-visemes__y_000_180_000__checkpoints__final',
 'Apple-visemes__z_000_000_120__checkpoints__final',
 'Apple-visemes__z_000_000_240__checkpoints__final',
 'Bed Single S__base_000_000_000__checkpoints__final',
 'Bed Single S__compound_090_000_090__checkpoints__final',
 'Bed Single S__x_180_000_000__checkpoints__final',
 'Bed Single S__y_000_180_000__checkpoints__final',
 'Bed Single S__z_000_000_120__checkpoints__final',
 'Bed Single S__z_000_000_240__checkpoints__final',
 'CarrotKhanStatue__base_000_000_000__checkpoints__final',
 'Car

In [4]:
invalid_tokens = []
invalid_positions = []

for mid in model_ids:
    tpath = os.path.join(token_dir, f"{mid}_tokens.pt")
    ppath = os.path.join(token_dir, f"{mid}_positions.pt")
    
    tokens = torch.load(tpath).float()       # [N_i, 2]
    positions = torch.load(ppath).float()    # [N_i, 3]
    
    # Prüfen, ob der Token-Tensor NaN oder Inf enthält
    if torch.isnan(tokens).any() or torch.isinf(tokens).any():
        invalid_tokens.append(mid)
    
    # Prüfen, ob der Positions-Tensor NaN oder Inf enthält
    if torch.isnan(positions).any() or torch.isinf(positions).any():
        invalid_positions.append(mid)

# Ausgabe
print("Model-IDs mit ungültigen Token-Werten (NaN/Inf):")
for mid in invalid_tokens:
    print("  ", mid)

print("\nModel-IDs mit ungültigen Positions-Werten (NaN/Inf):")
for mid in invalid_positions:
    print("  ", mid)


/tmp/ipykernel_25616/3247068785.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tokens = torch.load(tpath).float()       # [N_i, 2]
/tmp/ipykernel_25616/3247068785.py:9:

Model-IDs mit ungültigen Token-Werten (NaN/Inf):

Model-IDs mit ungültigen Positions-Werten (NaN/Inf):


In [5]:
import os
import torch

# Pfad und Modell-ID festlegen
token_dir = "prepared_objects_first_4_levels"
mid = "ATV-Truck-1__compound_090_000_090__checkpoints__final"

# Lade Token- und Positionsdaten
tpath = os.path.join(token_dir, f"{mid}_tokens.pt")
ppath = os.path.join(token_dir, f"{mid}_positions.pt")
tokens = torch.load(tpath).float()
positions = torch.load(ppath).float()

# Prüfen auf NaN/Inf in Tokens
nan_mask_tokens = torch.isnan(tokens)
inf_mask_tokens = torch.isinf(tokens)
nan_count_tokens = nan_mask_tokens.sum().item()
inf_count_tokens = inf_mask_tokens.sum().item()

# Prüfen auf NaN/Inf in Positions
nan_mask_positions = torch.isnan(positions)
inf_mask_positions = torch.isinf(positions)
nan_count_positions = nan_mask_positions.sum().item()
inf_count_positions = inf_mask_positions.sum().item()

# Sammle Indizes der betroffenen Zeilen
nan_indices_tokens = torch.nonzero(nan_mask_tokens, as_tuple=False)
inf_indices_tokens = torch.nonzero(inf_mask_tokens, as_tuple=False)
nan_indices_positions = torch.nonzero(nan_mask_positions, as_tuple=False)
inf_indices_positions = torch.nonzero(inf_mask_positions, as_tuple=False)

# Ausgabe der Ergebnisse
print(f"Modell: {mid}")
print(f"\n--- Tokens ---")
print(f"Shape: {tokens.shape}")
print(f"Anzahl NaN-Werte: {nan_count_tokens}")
print(f"Anzahl Inf-Werte: {inf_count_tokens}")
if nan_count_tokens > 0:
    print(f"Erste 5 NaN-Positionen (Index, Feature):")
    print(nan_indices_tokens[:5])
if inf_count_tokens > 0:
    print(f"Erste 5 Inf-Positionen (Index, Feature):")
    print(inf_indices_tokens[:5])

print(f"\n--- Positions ---")
print(f"Shape: {positions.shape}")
print(f"Anzahl NaN-Werte: {nan_count_positions}")
print(f"Anzahl Inf-Werte: {inf_count_positions}")
if nan_count_positions > 0:
    print(f"Erste 5 NaN-Positionen (Zeile, Spalte):")
    print(nan_indices_positions[:5])
if inf_count_positions > 0:
    print(f"Erste 5 Inf-Positionen (Zeile, Spalte):")
    print(inf_indices_positions[:5])

# Zeige beispielhafte betroffene Zeilen (max. 5) zur näheren Inspektion
if nan_count_tokens > 0 or inf_count_tokens > 0:
    affected_rows_tokens = torch.unique(nan_indices_tokens[:, 0]) if nan_count_tokens > 0 else torch.unique(inf_indices_tokens[:, 0])
    print(f"\nEinige Zeilen mit ungültigen Token-Werten (bis zu 5):")
    for idx in affected_rows_tokens[:5]:
        print(f"Index {idx.item()}: {tokens[idx].tolist()}")

if nan_count_positions > 0 or inf_count_positions > 0:
    affected_rows_positions = torch.unique(nan_indices_positions[:, 0]) if nan_count_positions > 0 else torch.unique(inf_indices_positions[:, 0])
    print(f"\nEinige Zeilen mit ungültigen Positions-Werten (bis zu 5):")
    for idx in affected_rows_positions[:5]:
        print(f"Index {idx.item()}: {positions[idx].tolist()}")


/tmp/ipykernel_25616/2593331568.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tokens = torch.load(tpath).float()


FileNotFoundError: [Errno 2] No such file or directory: 'prepared_objects_first_4_levels/ATV-Truck-1__compound_090_000_090__checkpoints__final_tokens.pt'